## Security Management In the Cloud: Security Group Rules

We look at how security for instances inside the Cloud can be managed via OpenStack's Security Group 

### Task 1:

- Deploy an instance inside OpenStack using Ubuntu 12.04 with a floating IP address
- Setup an FTP server inside this instance

** 1.1. Deploy instance using Ubuntu 12.04 **

- OpenStack cloud allows creation of instance using images loaded from external URL or uploaded from local machine
- OpenStack online image index: https://docs.openstack.org/image-guide/obtain-images.html
- Ubuntu: http://cloud-images.ubuntu.com/

<img src="figures/lab_2/01_image_list.png"/>

<img src="figures/lab_2/02_add_image.png"/>

<img src="figures/lab_2/03_launch_image_detail.png"/>

<img src="figures/lab_2/04_launch_image_source.png"/>

<img src="figures/lab_2/05_launch_image_flavor.png"/>

<img src="figures/lab_2/06_launch_image_network.png"/>

<img src="figures/lab_2/07_launch_image_network_port.png"/>

<img src="figures/lab_2/08_launch_image_security_group.png"/>

<img src="figures/lab_2/09_launch_image_key_pair.png"/>

<img src="figures/lab_2/10_launch_image_configuration.png"/>

<img src="figures/lab_2/11_launch_image_metadata.png"/>

<img src="figures/lab_2/12_launch_image_spawning.png"/>

<img src="figures/lab_2/13_launch_image_floating_ip_association.png"/>

<img src="figures/lab_2/14_launch_image_floating_ip_allocation.png"/>

<img src="figures/lab_2/15_launch_image_floatiing_ip_association_final.png"/>

We can deploy an instance directly from the `Instaces` tab

<img src="figures/lab_2/16_launch_instance.png"/>

** 1.2. Challenge: **

- Deploy a second instance inside OpenStack using the default `trusty-server` image and a floating IP address. 
- Name the second instance `test_1`
- Use the same configuration steps as `fpt_server`

** 1.3. Validate instance deployment: **

If the keys for `Key Pair` are set up correctly, you should be able to use your terminals to sssh into the `fpt_server` and `test_1` instances using their floating IP addresses and user account `ubuntu`. 

```
ssh ubuntu@<floating_ip>
```

You can also ssh into the CloudLab node that has `nmap` set up and perform a port scan on these two IP addresses to observer that SSH ports are opened:

<img src="figures/lab_2/17_validate_instances.png" />

** 1.4. Set up FTP server: **

Once your are logged into instance `fpt_server` via SSH, perform the following steps to set up an FTP server:

```
sudo su -
apt-get install -y vsftpd
```

Next, execute `vim /etc/vsftpd.conf` to modify the configuration of the FTP server. More specifically, make that the following lines are uncommended:

- listen=YES
- anonymous_enable=YES
- local_enable=YES
- write_enable=YES

Restart vsftpd service

```
service vsftpd restart
```

### Task 2:

- Modify default security group rules to allow FTP access

** 2.1. Check accessibility of the FTP server: **

Perform the following checks using *anonymous/anonymous* as FTP login and password:
- From `test_1`, ftp into `fpt_server` using the internal IP address of `fpt_server`
- From `test_1`, ftp into `fpt_server` using the floating IP address of `fpt_server`
- From your local terminal, ftp into `fpt_server` using the floating IP address of `fpt_server`
- Run `nmap` port scan on the floating IP address of `fpt_server`

By default, all instances are protected inside the Cloud Provider's security measures, isolated from outside world. Additional security configuration is needed to enable external access to the instances. In OpenStack, this can be done via editting security groups

<img src="figures/lab_2/18_default_security_group.png" />"

Within each security group, there are rules set in place to define how access can be done

<img src="figures/lab_2/19_default_security_group_rules.png" />"

We can add rules to the default security group to enable FTP access using TCP via port 20/21

<img src="figures/lab_2/20_default_security_group_add_rule.png" />"

Instances do not need to be modify in anyway. OpenStack will apply the new security rules immmediately on traffic coming to and going from instances associated with the modified security group

<img src="figures/lab_2/21_rule_impact.png" />"

** 2.2. Challenge: **

- Run nmap port scan on the floating IP addresses of `fpt_server` and `test_1`
- Identify a potential security breach?